In [93]:
import openai, os
from dotenv import load_dotenv
import shutil

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')
model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
client = openai.Client(api_key=openai_key)

## Crianto Storage Vector

In [94]:
# carregando todos os arquivos disponíveis da pasta data
files = os.listdir('data')
#filtar apenas os arquivos PDF
files = [file for file in files if file.endswith('.pdf')]
#verificar se o arquivo já existe na pasta data/generated com um sufixo _generated
files = [file for file in files if not os.path.isfile(f'data/generated/{file}_generated')]
#adicionar prefixo data aos arquivos
files = ['data/' + file for file in files]
print("Quantidade de arquivos a serem processados: ", len(files))
files


Quantidade de arquivos a serem processados:  1


['data/Comunicado - Catracas (tags).pdf']

In [95]:
def upload_file(filename):
    file = client.files.create(
        file=open(filename, "rb"),
        purpose="fine-tune"
    )
    return file

assistant_id = os.getenv('OPENAI_ASSISTANT_ID')
vector_store_id = os.getenv('OPENAI_VECTOR_STORE_ID')
print("ID do assistente: ", assistant_id)
print("ID do vector store: ", vector_store_id)



assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)
vector_store = client.beta.vector_stores.retrieve(vector_store_id)
print(vector_store)

ID do assistente:  asst_sJ7bR7ODfykH7DmOPCi29iar
ID do vector store:  vs_cHKbLWj0alqbfTyl1clP4LdL
Assistant(id='asst_sJ7bR7ODfykH7DmOPCi29iar', created_at=1724511079, description=None, instructions='Seu nome é Athena.\nO nome da faculdade é Instituto de Ensino Superior - iCEV\nNo primeiro contato, sempre Athena deve perguntar o nome da pessoa com quem está falando.\nObjetivo Geral:\nO chatbot da faculdade iCEV será responsável por responder perguntas dos alunos sobre o manual do estudante, rotinas acadêmicas, procedimentos administrativos, eventos, prazos, e outros processos relacionados à vida universitária.\n\n1. Escopo de Atendimento\nManuais e Normas:\nFornecer informações sobre o manual do estudante, incluindo normas de conduta, diretrizes acadêmicas, e procedimentos de avaliação.\nRotinas Acadêmicas:\nResponder sobre horários de aulas, calendário acadêmico, prazos de matrícula, e outros procedimentos relacionados ao semestre letivo.\nProcessos Administrativos:\nOrientar sobre pro

In [96]:
files

['data/Comunicado - Catracas (tags).pdf']

In [97]:
files_stream = [open(f, "rb") for f in files]
if len(files_stream)>0:
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id,
        files=files_stream
    )

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado

In [99]:

def copy_and_rename_file(source_path, destination_directory, new_name):
    # Define o caminho completo para o novo arquivo renomeado
    destination_path = os.path.join(destination_directory, new_name)
    print("Copiando de {} para {}".format(source_path, destination_path))
    # Copia o arquivo original para o novo destino com o novo nome
    shutil.copy2(source_path, destination_path)
    print(f"Arquivo copiado e renomeado para: {destination_path}")
    
for file in files:
    copy_and_rename_file(file, 'data/generated', file.replace("data/", "") + '_generated')

Copiando de data/Comunicado - Catracas (tags).pdf para data/generated\data/Comunicado - Catracas (tags).pdf_generated


FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado